# Test Environnement
Function to work with head(1000) or all the file

In [1]:

IS_TEST = True

# True: Testing with 1000 rows
# False: All data 50,000 rows


In [61]:

def get(df, is_test):
    if is_test is True:
        return df.sample(1000)
    else:
        return df

def normalize_X(df):
    mean = df.mean()
    std = df.std()
    result = (df - mean) / std
    print(mean.head())
    print(std.head())
    return result

# Read the training data

In [62]:
import pandas as pd
import numpy as np


print('Training set..')
df_train = get(pd.read_pickle('dataset/train_all.csv.p'), is_test = IS_TEST)
df_train.drop(['Unnamed: 0'], axis = 1, inplace = True)
print(df_train.head())
print(df_train.info())



Training set..
           eeg_0      eeg_1      eeg_2      eeg_3      eeg_4      eeg_5  \
47055   1.917578   7.234910  12.688800  18.198761  23.279926  27.314672   
23249  19.693089  19.982746  20.196733  20.144497  19.813780  19.315634   
34666  25.325607  26.432655  27.142454  26.932899  25.626194  23.457565   
26574 -35.796230 -34.179928 -32.442074 -30.616762 -28.808052 -27.121622   
48445  21.359604  19.890694  18.605400  18.015488  18.520382  20.076992   

           eeg_6      eeg_7      eeg_8      eeg_9       ...        \
47055  29.909950  31.313951  32.265450  33.702923       ...         
23249  18.822027  18.440676  18.236036  18.148605       ...         
34666  20.925425  18.627874  16.836210  15.417590       ...         
26574 -25.317991 -23.100096 -20.740799 -18.591986       ...         
48445  22.592400  25.748701  29.112122  32.409386       ...         

       respiration_z_398  respiration_x_399  respiration_y_399  \
47055           0.000131          -0.000095          

In [64]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.cross_validation import cross_val_score
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor

# Preparing the data

# PCA

### What is the best number of components for PCA ?

In [65]:
# define X_train, y_train:


X_train = normalize_X(df_train.drop(['power_increase'], axis=1))
y_train = df_train['power_increase']
    

eeg_0    0.201093
eeg_1    0.217411
eeg_2    0.211656
eeg_3    0.178919
eeg_4    0.121838
dtype: float64
eeg_0    37.371509
eeg_1    37.436841
eeg_2    37.524758
eeg_3    37.640044
eeg_4    37.779148
dtype: float64


In [66]:

print('Backup training set')
X = X_train
y = y_train



Backup training set


In [67]:
result = {}

print('Starting the loop..')

for k in [5, 10, 20, 30, 40, 50, 55, 60, 65, 70, 90]:
    print('Number of features: ', k)
    pca = PCA(n_components=k)
    X_pca = X
    X_pca = pca.fit_transform(X_pca)
    model = GradientBoostingRegressor()
    scores = cross_val_score(model, X_pca, y, cv=10, scoring='neg_mean_squared_error')
    result[k] = (-1) * scores.mean()
    print('Average score:', result[k])

Starting the loop..
Number of features:  5
Average score: 0.420685913947
Number of features:  10
Average score: 0.399651314801
Number of features:  20
Average score: 0.400678218103
Number of features:  30
Average score: 0.406508753381
Number of features:  40
Average score: 0.403441087513
Number of features:  50
Average score: 0.40168071206
Number of features:  55
Average score: 0.40290757779
Number of features:  60
Average score: 0.394171616869
Number of features:  65
Average score: 0.400581656871
Number of features:  70
Average score: 0.395463622991
Number of features:  90
Average score: 0.398786387359


We realize that the ideal value for n_component is between: 50 and 60 features

In [68]:
for k in range(50, 80):
    print('Number of features: ', k)
    pca = PCA(n_components=k)
    X_pca = X
    X_pca = pca.fit_transform(X_pca)
    model = GradientBoostingRegressor()
    scores = cross_val_score(model, X_pca, y, cv=10, scoring='neg_mean_squared_error')
    result[k] = (-1) * scores.mean()
    print('Average score:', result[k])
    


Number of features:  50
Average score: 0.405725806773
Number of features:  51
Average score: 0.398771007254
Number of features:  52
Average score: 0.40233795766
Number of features:  53
Average score: 0.402750491548
Number of features:  54
Average score: 0.399167413782
Number of features:  55
Average score: 0.397993551188
Number of features:  56
Average score: 0.403304432491
Number of features:  57
Average score: 0.398766693341
Number of features:  58
Average score: 0.393709473725
Number of features:  59
Average score: 0.402877737657
Number of features:  60
Average score: 0.39671157948
Number of features:  61
Average score: 0.398625676335
Number of features:  62
Average score: 0.404213454735
Number of features:  63
Average score: 0.398183574394
Number of features:  64
Average score: 0.404162700482
Number of features:  65
Average score: 0.405797700176
Number of features:  66
Average score: 0.406024837431
Number of features:  67
Average score: 0.405017172389
Number of features:  68
Averag

## Extract the right features

In [43]:
n_components_pca = 59

In [44]:
pca_final = PCA(n_components = n_components_pca)
# df_train = get(pd.read_pickle('dataset/train_all.csv.p'), is_test = False)
X_pca = pca_final.fit_transform(X_train)

### Comparing different Regression Models

In [12]:
from sklearn import ensemble

In [13]:
params = {'n_estimators': 200, 'max_depth': 5, 'min_samples_split': 2, 'learning_rate': 0.01, 'loss': 'ls'}
model = ensemble.GradientBoostingRegressor(**params)

In [14]:
scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')
print(scores)
print(scores.mean())

KeyboardInterrupt: 

In [15]:
model = LinearRegression()
scores = cross_val_score(model, X_pca, y, cv=10, scoring='neg_mean_squared_error')
print(scores)
print(scores.mean())

/usr/local/lib/python3.6/site-packages/scipy/linalg/basic.py:1018: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


[-0.4703154  -0.37468177 -0.41125781 -0.45417895 -0.37181151 -0.53628939
 -0.40685866 -0.46681844 -0.36533096 -0.41891772]
-0.427646060961


In [16]:
model = Lasso()
scores = cross_val_score(model, X_pca, y, cv=10, scoring='neg_mean_squared_error')
print(scores)
print(scores.mean())

[-0.47220814 -0.37074799 -0.406171   -0.45408559 -0.36902841 -0.53241903
 -0.41127672 -0.46850201 -0.37089952 -0.41807271]
-0.427341112919


In [18]:
model = Ridge()
scores = cross_val_score(model, X_pca, y, cv=10, scoring='neg_mean_squared_error')
print(scores)
print(scores.mean())

[-0.4703154  -0.37468177 -0.41125781 -0.45417895 -0.37181151 -0.53628939
 -0.40685866 -0.46681844 -0.36533096 -0.41891772]
-0.427646060728


In [19]:
model = ElasticNet()
scores = cross_val_score(model, X_pca, y, cv=10, scoring='neg_mean_squared_error')
print(scores)
print(scores.mean())

[-0.4710037  -0.37173003 -0.40776534 -0.45381173 -0.36943282 -0.53288969
 -0.40844486 -0.46763702 -0.36837259 -0.41799912]
-0.426908688199


In [20]:
model = DecisionTreeRegressor()
scores = cross_val_score(model, X_pca, y, cv=10, scoring='neg_mean_squared_error')
print(scores)
print(scores.mean())

[-0.91764421 -0.84181396 -0.90306196 -0.86026782 -0.81863279 -0.95800155
 -0.85357603 -0.85737621 -0.82208018 -0.76405428]
-0.859650898669


In [45]:
model = GradientBoostingRegressor()
scores = cross_val_score(model, X_pca, y, cv=10, scoring='neg_mean_squared_error')
print(scores)
print(scores.mean())

[-0.3966056  -0.40656972 -0.46533885 -0.313252   -0.34625264 -0.39982175
 -0.36686767 -0.3290872  -0.39876661 -0.38081037]
-0.380337240032


# Test

In [50]:
df_train = get(pd.read_pickle('dataset/train_all.csv.p'), is_test = False)
df_train.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [51]:
X_train = normalize_X(df_train.drop(['power_increase'], axis=1))
y_train = df_train['power_increase']

eeg_0    1.095197
eeg_1    1.132843
eeg_2    1.171287
eeg_3    1.207755
eeg_4    1.240513
dtype: float64
eeg_0    265.010375
eeg_1    273.592135
eeg_2    281.948289
eeg_3    290.226556
eeg_4    298.283547
dtype: float64


In [52]:
model = GradientBoostingRegressor()

In [53]:
X_train_pca = pca_final.transform(X_train)

In [54]:
model.fit(X_train_pca, y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

In [57]:
model.score(X_train_pca, y_train)

0.20180531107757183

Best : 0.20180531107757183

In [55]:
X_test = normalize_X(get(pd.read_pickle('dataset/test_all.csv.p'), is_test = False))
X_test.drop(['Unnamed: 0'], axis = 1, inplace = True)

Unnamed: 0    24999.500000
eeg_0            -0.449691
eeg_1            -0.453073
eeg_2            -0.458472
eeg_3            -0.464830
dtype: float64
Unnamed: 0    14433.901067
eeg_0            38.007630
eeg_1            37.965198
eeg_2            37.929855
eeg_3            37.899775
dtype: float64


In [58]:
X_test_pca = pca_final.transform(X_test)

In [59]:
y_predicted = model.predict(X_test_pca)

In [60]:
pd.DataFrame(y_predicted, index=np.arange(len(y_predicted)), columns=['power_increase']).to_csv('predictions/gbnormalisedreg.csv', index_label='index')